In [1]:
import os
import pandas as pd
import boto3

In [2]:
# Get AWS credentials from environment variables
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [3]:
# Set up S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [4]:
# Define the S3 bucket and CSV files
bucket_name = 'mindex-data-analytics-code-challenge'
bengals = 'bengals.csv'
boyd_receiving = 'boyd_receiving.csv'
chase_receiving = 'chase_receiving.csv'
higgins_receiving = 'higgins_receiving.csv'

In [5]:
# Download the CSV files
s3.download_file(bucket_name, bengals, bengals)
s3.download_file(bucket_name, boyd_receiving, boyd_receiving)
s3.download_file(bucket_name, chase_receiving, chase_receiving)
s3.download_file(bucket_name, higgins_receiving, higgins_receiving)

In [6]:
bengals_df = pd.read_csv(bengals)
boyd_receiving_df = pd.read_csv(boyd_receiving)
chase_receiving_df = pd.read_csv(chase_receiving)
higgins_receiving_df = pd.read_csv(higgins_receiving)

In [7]:
# Join and rename shared columns with player names in suffixes
result_df = pd.merge(bengals_df, boyd_receiving_df, on='Week', how='inner')
result_df = pd.merge(result_df, chase_receiving_df, on='Week', how='inner', suffixes=('_boyd', '_chase'))
result_df = pd.merge(result_df, higgins_receiving_df, on='Week', how='inner')
result_df = result_df.rename(columns={'Yards_boyd': 'boyd_yards', 'TD_boyd': 'boyd_td'})
result_df = result_df.rename(columns={'Yards_chase': 'chase_yards', 'TD_chase': 'chase_td'})
result_df = result_df.rename(columns={'Yards': 'higgins_yards', 'TD': 'higgins_td'})

# replace game 'Result' values 0.0 and 1.0 with 'Loss' and 'Win', respectively
result_df['Result'] = result_df['Result'].replace({0.0: 'Loss', 1.0: 'Win'})

In [8]:
result_df.head()

,Week,Opponent,Location,Result,boyd_yards,boyd_td,chase_yards,chase_td,higgins_yards,higgins_td
0,REG1,MIN,Home,Win,32,0,101,1,58,1
1,REG2,CHI,Away,Loss,73,0,54,1,60,1
2,REG5,GB,Home,Loss,24,0,159,1,32,0
3,REG6,DET,Away,Win,7,0,97,0,44,0
4,REG7,BAL,Away,Win,39,0,201,1,62,0


In [9]:
from sqlalchemy import create_engine

# Define the connection parameters
host = os.environ.get('POSTGRES_HOST')
user = os.environ.get('POSTGRES_USER')
password = os.environ.get('POSTGRES_PASSWORD')
dbname = 'postgres'
table_name = 'alex_jansing'

# Create a connection string
conn_str = f'postgresql://{user}:{password}@{host}/{dbname}'

# Create a SQLAlchemy engine
engine = create_engine(conn_str)

In [10]:
# Write the DataFrame to the PostgreSQL database
result_df.to_sql(table_name, engine, if_exists='replace', index=False)

18

In [11]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class YourModel(Base):
    __tablename__ = table_name
    week = Column(String, primary_key=True, name="Week")
    opponent = Column(String, name="Opponent")
    location = Column(String, name="Location")
    result = Column(String, name="Result")
    boyd_yards = Column(Integer)
    boyd_td = Column(Integer)
    chase_yards = Column(Integer)
    chase_td = Column(Integer)
    higgins_yards = Column(Integer)
    higgins_td = Column(Integer)

/var/folders/4z/vcgsc1ks04d_gy30fqd48thr0000gn/T/ipykernel_45726/1999666857.py:4: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [12]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

result = engine.execute('''
    select
	sum("boyd_yards") as "Boyd Yards", sum("higgins_yards") as "Higgins Yards", sum("chase_yards") as "Chase Yards",
	COUNT(CASE WHEN "Result" = 'Win' THEN 1 END) || '-' || 
    COUNT(CASE WHEN "Result" = 'Loss' THEN 1 END) as "Win/Loss"
from alex_jansing aj ;
''')

In [13]:
# Fetch and print the results
for row in result:
    print(row)

(Decimal('784'), Decimal('1400'), Decimal('1655'), '11-7')


# SQL Query

```sql
select
	sum("boyd_yards") as "Boyd Yards", sum("higgins_yards") as "Higgins Yards", sum("chase_yards") as "Chase Yards",
	COUNT(CASE WHEN "Result" = 'Win' THEN 1 END) || '-' || 
    COUNT(CASE WHEN "Result" = 'Loss' THEN 1 END) as "Win/Loss"
from alex_jansing aj ;
```

![SQL Results](dbeaver.png)
For some reason, not all of the yarns match the "Mindex Data Analytics Code Challenge" document, but perhaps this is on purpose?